# Consumption at Home

ℹ️   Analyse consumption of Gas / Water and Electricity at home

Before start, make sure you rename `config.ini.example` into `config.ini` and adjust all the config's values.

❗️  This will download CSV file from the remove url. Checkout [this guide](https://www.labnol.org/internet/direct-links-for-google-drive/28356/#google-sheets---export-links) if you wish to get the proper link to download CSV directly from Google Sheets.

In [ ]:
import configparser

# Load settings

config = configparser.ConfigParser()
config.read('config.ini')
config.sections()

typeColumn = config['MAIN']['TypeColumnName']
timeColumn = config['MAIN']['TimeColumnName']
valueColumn = config['MAIN']['ValueColumnName']
daysAgoRange = config['MAIN']['daysAgoRange']
csvUrl = config['MAIN']['FileUrl']

print('Config was loaded.')

In [ ]:
import pandas as pd
import requests
import io

# Download CSV

r = requests.get(csvUrl)
if r.ok:
    data = r.content.decode('utf8')
    df = pd.read_csv(io.StringIO(data), dayfirst=True, parse_dates=True)
    print("Data downloaded!")

In [ ]:
# Drop hidden columns
# NB: Feel free to comment these lines out. This was necessary for me because I had that in my CSV file
del df['Дата']
del df['Пометка']

In [ ]:
# Check out what we have
df.info()

In [ ]:
# and the data
df.head()

In [ ]:
# let's convert timeColumn into date format to work with later
df[timeColumn] = pd.to_datetime(df[timeColumn], dayfirst=True)
df.info()

In [ ]:
# and check that all the dates were converted properly
df

In [ ]:
# Select consumption type
consumptionType = input("Select a type:")
print("Chosen type is " + consumptionType)

In [ ]:
# Filter out data by chosen type
dataByType = df[df[typeColumn] == consumptionType]

In [ ]:
print("Examples of records in the filtered dataset:")
dataByType.head()

In [ ]:
from datetime import date
from datetime import datetime as dtObj
import datetime

# filter for daysAgoRange days ago
daysAgo = date.today() - datetime.timedelta(days=int(daysAgoRange))
today = date.today()
dt = dtObj(daysAgo.year, daysAgo.month, daysAgo.day)
dtToday = dtObj(today.year, today.month, today.day)
print('Filter data for the last', daysAgoRange, 'day(s), from', dt, 'to', dtToday)

# Filter out for the dates
dateFilterdDf = dataByType[(dataByType[timeColumn] >= dt) & (dataByType[timeColumn] <= dtToday)]
dateFilterdDf

In [ ]:
# Convert dates into strings and make them more readable
#   so that we can just display them in the plot later
def format_date(x):
    return x.strftime("%b %y")

dateFilterdDf[timeColumn] = dateFilterdDf[timeColumn].apply(format_date)
dateFilterdDf

In [ ]:
# Drop unneeded column with type
del dateFilterdDf[typeColumn]

In [ ]:
# Group by month and select min of each group to get 1 record per month to work with
# We take min to make sure we take the closest record to the beginning of the month 
idx = dateFilterdDf.groupby([timeColumn], sort=False)[valueColumn].transform(min) == dateFilterdDf[valueColumn]
grouppedDf = dateFilterdDf[idx]
grouppedDf

In [ ]:
# To make sure we have all months sorted correctly, we sort it by valueColumn
# We do so as we know that counter value is always a cummulative sum, so it always growths.
sortedDf = grouppedDf.sort_values(by=[valueColumn])
sortedDf

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Because all the values are a cummulative sum, we extract diff between neighbours
# and set the index to move sure the .diff will not complain about strings in timeColumn
diffDf = sortedDf.set_index(timeColumn).diff()
diffDf

In [ ]:
# Remove index to make it easier to draw a plot
diffDfWithoutIndex = diffDf.reset_index(level=0)
diffDfWithoutIndex

In [ ]:
# Drop first row that is NaN anyway and so has no any valuable data for us.
#dfWithoutFirstAndLastRows = diffDfWithoutIndex.drop(diffDfWithoutIndex.tail(1).index) # drop last n rows
dfWithoutFirstRow = diffDfWithoutIndex.drop(diffDfWithoutIndex.head(1).index) # drop first n rows
dfWithoutFirstRow

In [ ]:
# Draw the plot
ax = dfWithoutFirstRow.plot(x = timeColumn, y = valueColumn, figsize=(15,7))

# Make sure we display all the dates in X
ax.set_xticks(list(range(0, len(dfWithoutFirstRow[timeColumn]))))
ax.set_xticklabels(dfWithoutFirstRow[timeColumn]);